In [ ]:
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import matplotlib.dates
from datetime import datetime

### Total active cases

In [ ]:
list_files = sorted(os.listdir('Data/'))
list_files = [file for file in list_files if '2021' in file]
def sum_col(col_name):
    result = []
    for file_name in list_files:
        df = pd.read_csv('Data/'+file_name,encoding='utf-8',sep=';')
        df = df.fillna(0)
        result.append(df[col_name].sum()/1000)
    return result
new_cases = sum_col('New Cases')
active_cases = sum_col('Active Cases')

In [ ]:
list_files = sorted(os.listdir('Data/'))
list_files = [file for file in list_files if '2021' in file]

periods =  [datetime.strptime(x.split('.')[0],'%Y-%m-%d') for x in list_files]
periods = [x.strftime('%d-%m') for x in periods]

plt.plot(periods,active_cases)
plt.ylabel('Số ca bệnh active (nghìn người)')
plt.xlabel('Ngày')
plt.title('Tổng số ca active trên thế giới')
plt.xticks(rotation=60)
plt.show()

plt.title('Tổng số lượng ca mới')
plt.xticks(rotation=60)
plt.ylabel('Số ca bệnh active (nghìn người)')
plt.xlabel('Ngày')
plt.plot(periods, new_cases)
plt.show()

In [ ]:
periods =  [datetime.strptime(x.split('.')[0],'%Y-%m-%d') for x in list_files]
periods = [x.strftime('%d-%m') for x in periods]

def sum_col_region(col_name,region):
    result = []
    for file_name in list_files:
        df = pd.read_csv('Data/'+file_name,encoding='utf-8',sep=';')
        df['Continent'] = df['Continent'].fillna('Unknow')
        df = df.fillna(0)
        Total = df[df['Continent']==region][col_name].sum()/1000
        result.append(Total)
    return result

file_name = 'Data/2021-11-28.csv'
df = pd.read_csv(file_name,encoding='utf-8',sep=';')
df['Continent'] = df['Continent'].fillna('Unknow')
regions = set(df['Continent'])
dic_region = {}

data = pd.DataFrame([],columns = ['Region']+periods)

for region in regions:
    if(region in ['Unknow']):
        continue
    dic_region[region] = sum_col_region('Active Cases',region)
    data.loc[len(data)] = [region]+dic_region[region]
    plt.plot(periods,dic_region[region],label=region)
plt.legend(loc='best')
plt.title("Active cases")
plt.ylabel("Số lượng ca (nghìn ca)")
plt.xlabel('Ngày')
plt.xticks(rotation=60)
plt.savefig('RegionActive.jpg')
plt.show()

pd.options.display.float_format = '{:,.0f}'.format
data

#### New case in regions

In [ ]:
for region in regions:
    dic_region[region] = sum_col_region('New Cases',region)
    plt.plot(periods,dic_region[region],label=region)
plt.legend(loc='best')
plt.ylabel("Số lượng ca (nghìn ca)")
plt.xlabel('Ngày')
plt.xticks(rotation=60)
plt.title("New cases")
plt.show()
for region in regions:
    dic_region[region] = sum_col_region('New Recovered',region)
    plt.plot(periods,dic_region[region],label=region)
plt.legend(loc='best')
plt.ylabel("Số lượng ca (nghìn ca)")
plt.xlabel('Ngày')
plt.title("New recovered")
plt.xticks(rotation=60)
plt.show()

### Chúng ta sẽ xem xét tiếp tục cột Total Deaths
Ở cột này, chúng ta sẽ lấy dữ liệu mới nhất của từng vùng để xem xét

In [ ]:
list_files = sorted(os.listdir('Data/'))
file_name = "Data/"+list_files[len(list_files)-1]
df = pd.read_csv(file_name,encoding='utf-8',sep=';')
df['Continent'] = df['Continent'].fillna('Unknow')
regions = set(df['Continent'])
x = []
y = []
fig = plt.figure(figsize=(10,1))
fig.add_axes([0,0,2,6])
for region in regions:
    x.append(region)
    y.append(df[df['Continent']==region]['Total Deaths'].sum()/1000)
plt.ylabel('Khu vực')
plt.xlabel('Số người chết (nghìn người)')
plt.title("Tổng số người chết")
plt.barh(x,y)
plt.show()


### Total deaths in countries

In [ ]:
list_files = sorted(os.listdir('Data/'))
file_name = "Data/"+list_files[len(list_files)-1]
pd.options.display.float_format = '{:,.10f}'.format
df = pd.read_csv(file_name,encoding='utf-8',sep=';')
df['Continent'] = df['Continent'].fillna('Unknow')
groups = list(df.groupby(['Continent']).groups.keys())
df = df.groupby(['Continent']).sum()

df['Portion'] = df['Total Deaths']*100/df['Total Cases']
df['Portion']
for i in range(len(df)):
    plt.pie([df['Portion'][i],100-df['Portion'][0]],labels=['Deaths','Not deaths'],autopct='%1.2f%%')
    plt.title(f'{groups[i]}')
    plt.show()
df = df.sort_values(by=['Portion'])
df['Portion']

In [ ]:
list_files = sorted(os.listdir('Data/'))
file_name = "Data/"+list_files[len(list_files)-1]
df = pd.read_csv(file_name,encoding='utf-8',sep=';')
df['Continent'] = df['Continent'].fillna('Unknow')
regions = set(df['Continent'])
# fig = plt.figure(figsize=(10,1))
fig,(ax1,ax2,ax3) = plt.subplots(nrows=3,ncols=1,figsize=(20,15))
# plt.xlabel('Khu vực')
for tick in ax1.get_xticklabels():
    tick.set_rotation(90)
for tick in ax2.get_xticklabels():
    tick.set_rotation(90)
for tick in ax3.get_xticklabels():
    tick.set_rotation(90)
df = df.sort_values(by=['Total Deaths'],ascending=False)
df = df[df['Total Deaths']>10000]
# ax2.bar(df['Country,Other'],df['Total Deaths'])
ax1.bar(df['Country,Other'],df['Deaths/1M pop'])
ax1.set_xlabel('Nước')
ax1.set_ylabel('Số người chết')
ax1.set_title('Deaths/1M population')


ax2.bar(df['Country,Other'],df['Tot Cases/1M pop']/1000)
ax2.set_xlabel('Nước')
ax2.set_ylabel('Số ca (nghìn)')
ax2.set_title('Total/1M population')


ax3.bar(df['Country,Other'],df['Tests/1M pop']/1000)
ax3.set_xlabel('Nước')
ax3.set_ylabel('Số lượt test (nghìn)')
ax3.set_title('Tests/1M pop')

fig.tight_layout()
fig.savefig('group.jpg')
plt.show()

